In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os 
import pandas as pd 
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X.values).reshape(len(X),1,28,28)
        self.y = torch.tensor(y.values)
        
    def __getitem__(self, idx):
        img = self.X[idx] / 255
        label = self.y[idx]
        return img,label
    
    def __len__(self):
        return len(self.y)

In [ ]:
class TestDataset(Dataset):
    def __init__(self, X):
        self.X = torch.tensor(X.values).reshape(len(X),1,28,28)
        
    def __getitem__(self, idx):
        img = self.X[idx] / 255
        return img
    
    def __len__(self):
        return len(self.X)

In [ ]:
df = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
X = df.loc[:,df.columns  != 'label']
y = df['label']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.1,random_state = 123)
train_set = CustomDataset(X_train,y_train)
test_set = CustomDataset(X_test,y_test)

In [ ]:
print('Length of train_set:', len(train_set))
print('Length of test_set:', len(test_set))

In [ ]:
X_test_csv = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
pred_set = TestDataset(X_test_csv)

In [ ]:
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# Hyper parameters
num_epochs = 100
batch_size = 64
learning_rate = 0.001
train_loader = DataLoader(dataset = train_set, batch_size = batch_size, shuffle=True)
test_loader = DataLoader(dataset = test_set,batch_size = batch_size, shuffle = False)


In [ ]:
pred_loader = DataLoader(dataset = pred_set,batch_size = batch_size, shuffle = False)

In [ ]:
images,labels = train_set[0]
plt.imshow(images[0])

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, channel):
        super().__init__()
        self.conv1 = nn.Conv2d(channel, channel, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(channel, channel, kernel_size=3, padding=1)

    def forward(self, x):
        y = F.relu(self.conv1(x))
        y = self.conv2(y)

        return F.relu(x + y)

class ConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(         
            nn.Conv2d(1,32,5,1,2),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            ResidualBlock(32),
            nn.MaxPool2d(kernel_size=2),
            
            nn.Dropout2d(0.3),
            nn.Conv2d(32,32,5,1,2),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            ResidualBlock(32),
            nn.MaxPool2d(kernel_size=2),
            
            nn.Dropout2d(0.3),
            nn.Conv2d(32,16,5,1,2),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            ResidualBlock(16),
            nn.MaxPool2d(kernel_size=2),
            
            nn.Dropout2d(0.3),
            nn.Conv2d(16,8,5,1,2),
            nn.ReLU(),
            nn.Flatten()
        )
        self.fc = nn.Linear(72, 10)
        
    def forward(self, x):
        out = self.conv(x)
        out = self.fc(out)
        # out = F.softmax(out, dim=1)
        return  out

In [ ]:
model = ConvNet().to(device)


In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    per_epoch_loss = 0
    train_loss = 0
    
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        train_loss += loss.item()
        per_epoch_loss += loss.item()
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 50 == 0:
            train_loss /= 50
            print (f'Epoch [{epoch + 1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {train_loss:.6f}')
            train_loss = 0
            
    print(f'Epoch:[{epoch + 1}/{num_epochs}], Average Loss: {per_epoch_loss/total_step:.6f}')

In [ ]:
model.eval()
true_label = []
pred_label = []
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        pred_label.extend(torch.argmax(outputs, axis = 1).cpu().numpy())
        true_label.extend(labels.cpu().numpy())

cf_mat = confusion_matrix(true_label, pred_label)
clf_report = classification_report(true_label, pred_label,digits = 4)
print(cf_mat)
print(clf_report)

In [ ]:
pred_label = []
with torch.no_grad():
    for images in pred_loader:
        images = images.to(device)
        outputs = model(images)
        pred_label.extend(torch.argmax(outputs, axis = 1).cpu().numpy())

In [ ]:
output = pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')
output['Label'] = pred_label
output.to_csv('submission.csv', index=False)

In [ ]:
output.head()